In [1]:
import tensorflow as tf

数据集加载

In [2]:
from tensorflow import keras
(x,y),(x_test,y_test) = keras.datasets.mnist.load_data()
type(x)

numpy.ndarray

In [3]:
db = tf.data.Dataset.from_tensor_slices(x_test)#将numpy转换成tensor,可以多参数
db = db.shuffle(100)
next(iter(db)).shape

TensorShape([28, 28])

In [4]:
# 数据预处理
db.map(print)#每个图片执行print函数

<MapDataset element_spec=TensorSpec(shape=<unknown>, dtype=tf.int32, name=None)>

In [5]:
# 数据取片
db1 = db.batch(23)
db_iter = iter(db1)
next(db_iter).shape

TensorShape([23, 28, 28])

In [6]:
# 迭代多次
db2 = db1.repeat()#永远不会迭代结束
db3 = db1.repeat(2)#迭代两次就介绍

全链接层

In [9]:
#  构造【n，y】的网络层，n有输入确定，y有输出确定
net1 = tf.keras.layers.Dense(10)
input = tf.random.truncated_normal([256,125])
out = net1(input)#前向计算一层网络
out.shape
print(net1.kernel[0,:])
net1.bias
 

tf.Tensor(
[ 0.19179626  0.13661523 -0.16143602  0.17120083  0.20744158 -0.14924833
 -0.07402708 -0.07320069 -0.10700993  0.18347205], shape=(10,), dtype=float32)


<tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
net = tf.keras.layers.Dense(12)
net.build(input_shape=[None,123])
net.build(input_shape=[None,22])#构造隐藏层的形状
print(net.kernel)

In [12]:
# 构造网络
model = tf.keras.Sequential(
    [keras.layers.Dense(2,activation="relu"),keras.layers.Dense(2,activation="relu"),keras.layers.Dense(2,activation="relu")]
)
model.build(input_shape=[None,12])
model.summary()
for p in model.trainable_variables:
    print(p.name,p.shape)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2)                 26        
                                                                 
 dense_5 (Dense)             (None, 2)                 6         
                                                                 
 dense_6 (Dense)             (None, 2)                 6         
                                                                 
Total params: 38
Trainable params: 38
Non-trainable params: 0
_________________________________________________________________
dense_4/kernel:0 (12, 2)
dense_4/bias:0 (2,)
dense_5/kernel:0 (2, 2)
dense_5/bias:0 (2,)
dense_6/kernel:0 (2, 2)
dense_6/bias:0 (2,)


#### 输出方式  
linear or naive classfication with mse or other general prediction:  y->[-无穷大 +无穷大]  
二分类 图像生成: y-> [0 1]  
激活函数：relu y->[0 +无穷大]  tf.sigmoid y->[0 1]  
多分类：y->[0 1] and sum(y)=1
激活函数：tf.nn.softmax()
y->[-1 1] tf.tanh() 



In [ ]:
a = tf.linspace(-6,6,10)
print(a)
b=tf.sigmoid(a)
tf.reduce_min(b),tf.reduce_max(b)

In [16]:
tf.nn.softmax(a),tf.tanh(a)

(<tf.Tensor: shape=(10,), dtype=float64, numpy=
 array([4.52462289e-06, 1.71649166e-05, 6.51179930e-05, 2.47036039e-04,
        9.37172691e-04, 3.55532195e-03, 1.34877107e-02, 5.11678952e-02,
        1.94114001e-01, 7.36404055e-01])>,
 <tf.Tensor: shape=(10,), dtype=float64, numpy=
 array([-0.99998771, -0.99982316, -0.99745797, -0.96402758, -0.58278295,
         0.58278295,  0.96402758,  0.99745797,  0.99982316,  0.99998771])>)

##### 误差计算  
MSE: loss=sum((y-out)^2)/n tf.losses.MSE() 返回一个[b]的向量，要求均值  
交叉熵：熵越小，信息量越大，越不稳定  
多分类  H(p,q)=-sum(p*log(q))=-1*log(q1),p1 != 0,p为真实值，q为预测值  
当使用one-hot编码时，p=q，H(p,q)=0,意味着预测与真实值相等  
单分类 loss = -（p*log(q)+(1-p)*log(1-q)  
tf.losses.categorical_crossentropy(p,q)

In [18]:
# 熵计算
a=tf.constant([0.2,0.4,0.1,0.29,0.01])
tf.reduce_sum(a*tf.math.log(a)/tf.math.log(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=-1.9096919>

In [24]:
# 交叉熵计算
tf.losses.categorical_crossentropy([0,1,0,0],[0.1,0.4,0.25,0.25],from_logits=True)
# 二分类交叉熵
# tf.losses.binary_crossentropy([1],[0.9])

<tf.Tensor: shape=(), dtype=float32, numpy=1.2419142>